In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터셋, 데이터 로더 생성

In [2]:
dataset = FashionMNIST(root='./data', train=True, transform=ToTensor())
train_set, val_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(val_set, batch_size=100, shuffle=False)

#이미지 넘어오는거 확인
#for i, e in train_loader:
   #print(i,e)

# 모델 생성

In [3]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self,x):
        out = self.linear(x)
        return out

# 하이퍼 파라미터 설정

In [4]:
input_size = 28*28
num_classes = 10
num_epoches = 100
learning_rate = 0.001
num_models = 5 # 앙상블 모델의 수

# 앙상블

In [5]:
# 앙상블 모델 생성
models = [LogisticRegression(input_size, num_classes) for _ in range(num_models)]

criterion = nn.CrossEntropyLoss()
optimizer = [optim.SGD(model.parameters(), lr=learning_rate) for model in models]

# 학습

In [6]:
for epoch in range(num_epoches):
    for i, (imgs, labels) in enumerate(train_loader):
        #이미지 로드
        imgs = imgs.reshape(-1, input_size)
        labels = labels
        
        for j in range(num_models):
            outputs = models[j](imgs)
            loss = criterion(outputs, labels)
            
            optimizer[j].zero_grad()
            loss.backward()
            optimizer[j].step()
            
    # 검증
    with torch.no_grad():
        total, correct = 0, 0
        for imgs, labels in test_loader:
            imgs = imgs.reshape(-1,input_size)
            outputs = torch.zeros(imgs.size()[0], num_classes) # 입력 배치에 대한 출력 텐서 초기화
            
            # 앙상블 모델의 예측값 더하가
            for j in range(num_models):
                outputs += models[j](imgs)
                _,predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted==labels).sum().item()
                
        val_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epoches}], Validation Acc: {val_acc:.2f}%')

Epoch [1/100], Validation Acc: 64.25%
Epoch [2/100], Validation Acc: 65.87%
Epoch [3/100], Validation Acc: 66.75%
Epoch [4/100], Validation Acc: 67.65%
Epoch [5/100], Validation Acc: 68.50%
Epoch [6/100], Validation Acc: 69.55%
Epoch [7/100], Validation Acc: 70.69%
Epoch [8/100], Validation Acc: 71.40%
Epoch [9/100], Validation Acc: 72.16%
Epoch [10/100], Validation Acc: 72.89%
Epoch [11/100], Validation Acc: 73.42%
Epoch [12/100], Validation Acc: 73.92%
Epoch [13/100], Validation Acc: 74.31%
Epoch [14/100], Validation Acc: 74.77%
Epoch [15/100], Validation Acc: 74.89%
Epoch [16/100], Validation Acc: 75.42%
Epoch [17/100], Validation Acc: 75.82%
Epoch [18/100], Validation Acc: 76.26%
Epoch [19/100], Validation Acc: 76.41%
Epoch [20/100], Validation Acc: 76.70%
Epoch [21/100], Validation Acc: 76.92%
Epoch [22/100], Validation Acc: 77.24%
Epoch [23/100], Validation Acc: 77.41%
Epoch [24/100], Validation Acc: 77.64%
Epoch [25/100], Validation Acc: 77.79%
Epoch [26/100], Validation Acc: 77